<a href="https://colab.research.google.com/github/josesg998/competencia_properati/blob/main/TP_Individual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

In [11]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Lectura de datos

In [ ]:
#DIR = "/content/drive/MyDrive/Data Mining Properati/"

#df_train = pd.read_feather(DIR + 'properati_entrenamiento.ftr').set_index('id')
#df_test = pd.read_feather(DIR + "properati_a_predecir.ftr").set_index('id')

In [14]:
#df_train = pd.read_csv(DIR + "properati_entrenamiento.zip", index_col="id")  
#df_train.head()

# df_test = pd.read_csv(DIR + "properati_a_predecir.zip", index_col="id")  
#df_test.head()

# df_train = pd.read_csv('bases/properati_entrenamiento.csv',index_col='id')
# df_test = pd.read_csv('bases/properati_a_predecir.csv',index_col='id')
# df_test.head()

# df_train.reset_index(drop=False).to_feather('bases/properati_entrenamiento.ftr')
# df_test.reset_index(drop=False).to_feather('bases/properati_a_predecir.ftr')

In [ ]:
df_train = pd.read_feather('bases/properati_entrenamiento.ftr').set_index('id')
df_test = pd.read_feather('bases/properati_a_predecir.ftr').set_index('id')
df_train.head(3)

,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
id,,,,,,,,,,,,,,,,,,,,,
1,Propiedad,2019-07-07,2019-07-10,2019-07-07,NaN,NaN,Argentina,Buenos Aires Interior,Urquiza,Urquiza,...,NaN,NaN,NaN,3800.0,ARS,Mensual,COCHERA EN ALQUILER,Xintel (SLC-SI1-560) Alquiler de Cochera en CA...,Cochera,Alquiler
2,Propiedad,2019-07-07,9999-12-31,2019-07-07,-64.184812,-31.421307,Argentina,Córdoba,Córdoba,None,...,NaN,12.0,12.0,16000.0,USD,Mensual,Cochera venta,VENDO COCHERA- EXCELENTE UBICACIÓN! Calle Buen...,Cochera,Venta
3,Propiedad,2019-07-07,2019-07-09,2019-07-07,-57.965880,-34.902568,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,La Plata,...,NaN,12.0,NaN,16000.0,USD,None,Cochera cubierta con portón automatizado en ve...,Cochera Cubierta en venta ubicada en calle 7 e...,Cochera,Venta


# Entendimiento (magia parte 1 -- AID)

# Limpieza y transformación de datos (magia parte 2 -- MD)

In [15]:
# Selección de datos. Solo a fines demostrativos. Cambiar!
df_train = df_train.loc[(df_train.l2 == "Córdoba") & (df_train.operation_type == 'Venta') & (df_train.property_type == 'Casa')]
df_train.shape

(11100, 24)

In [ ]:
## esto tiene que hacerse antes y mejor
df_train.fillna(0, inplace=True, downcast= "infer")
df_test.fillna(0, inplace=True, downcast= "infer") 

In [ ]:
# Funcion de limpieza (usada en la siguiente sección) que usa y_train
def limpiar_fold(X_train, y_train, X_test):
    ## TODO: limpiar los datos

    return (X_train, y_train, X_test)

# Entrenamiento del modelo

## Territorio de Aprendizaje Automático
☢☢☢☢☢☢ Se mira y no se toca ☢☢☢☢☢☢☢

In [ ]:
# Datos para probar
df_train = df_train.select_dtypes(include=['float64', 'int64', 'int32', 'int16', 'int8', 'bool'])

X = df_train[df_train.columns.drop('price')]
y = df_train['price']

In [ ]:
# Creamos el modelo
reg = sk.ensemble.RandomForestRegressor(n_estimators=500, max_depth=5, n_jobs=-1, random_state=42)

# Partimos en entrenamiento+prueba y validación
X_train_test, X_val, y_train_test, y_val = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

scores_train = []
scores_test = []
# Validación cruzada, 10 folds, shuffle antes
kf = sk.model_selection.KFold(n_splits=10, shuffle=True, random_state=42)
for fold, (train_index, test_index) in enumerate(kf.split(X_train_test, y_train_test)):
    X_train, X_test, y_train, y_test = X_train_test.iloc[train_index], X_train_test.iloc[test_index], y_train_test.iloc[train_index], y_train_test.iloc[test_index]

    X_train, y_train, X_test = limpiar_fold(X_train, y_train, X_test)

    # Entrenamos el modelo
    reg.fit(X_train, y_train)

    # Predecimos en train
    y_pred = reg.predict(X_train)

    # Medimos la performance de la predicción en test
    score_train = sk.metrics.mean_squared_error(y_train, y_pred)
    scores_train.append(score_train)

    # Predecimos en test
    y_pred = reg.predict(X_test)    

    # Medimos la performance de la predicción en test
    score_test = sk.metrics.mean_squared_error(y_test, y_pred)
    scores_test.append(score_test)
    
    print(f"{fold=}, {score_train=} {score_test=}")
    

print(f"Train scores mean={pd.Series(scores_train).mean()}, std={pd.Series(scores_train).std()}")
print(f"Test scores mean={pd.Series(scores_test).mean()}, std={pd.Series(scores_test).std()}")

NameError: name 'limpiar_fold' is not defined

# Predicción para kaggle


In [ ]:
## Datos a predecir 
X = df_train[df_train.columns.drop('price')]
y = df_train['price']
X_prueba = df_test[df_train.columns.drop('price')] # cuidado:

# Entrenamos el modelo con todos los datos
reg.fit(X, y)

# Predecimos
df_test['price'] = reg.predict(X_prueba)

# Grabamos
df_test['price'].to_csv('solucion.csv', index=True)

In [ ]:
pd.DataFrame(zip(df_train.columns.drop('price'), reg.feature_importances_), columns=["column", "feature_importance"]).sort_values(by="feature_importance", ascending=False)

,column,feature_importance
1,lon,0.284658
0,lat,0.212267
4,rooms,0.141806
8,surface_covered,0.125101
5,bedrooms,0.095744
7,surface_total,0.073814
6,bathrooms,0.066611
2,l5,0.000000
3,l6,0.000000
